# Load and Save Files for CanRCM4 Downscaling Project
Load in and process HRDPS files here then export them as netcdf or numpy arrays<br>
This file is for the older HRDPS data

In [1]:
from pathlib import Path
import time
#import datetime as dt #only keep if adding the metadata near the end is fixed

import numpy as np
import xarray as xr

In [2]:
hrdps_path = Path("/results/forcing/atmospheric/GEM2.5/gemlam/")

In [28]:
# hrdps =xr.open_mfdataset(hrdps_path/"gemlam_y2008m05d*.nc")
hrdpsm10d09 =xr.open_mfdataset(sorted(hrdps_path.glob(f"gemlam_y2010m10d09.nc")))
hrdpsm10d10=xr.open_mfdataset(sorted(hrdps_path.glob(f"gemlam_y2010m10d10.nc")))
hrdpsm10d11=xr.open_mfdataset(sorted(hrdps_path.glob(f"gemlam_y2010m10d11.nc")))
hrdpsm10d12=xr.open_mfdataset(sorted(hrdps_path.glob(f"gemlam_y2010m10d12.nc")))

Processing all of the years at once with a single open_mfdataset() call was slow and had a huge memory footprint. So process 1 year at a time, storing the years in a dict, then concatenate them near the end of the processing.

In [30]:
hrdpsm10d09.time_counter
# hrdpsm10d10.time_counter
# hrdpsm10d11.time_counter
# hrdpsm10d12.time_counter

<xarray.DataArray 'time_counter' (time_counter: 24)>
array(['2010-10-09T00:00:00.000000000', '2010-10-09T01:00:00.000000000',
       '2010-10-09T02:00:00.000000000', '2010-10-09T03:00:00.000000000',
       '2010-10-09T04:00:00.000000000', '2010-10-09T05:00:00.000000000',
       '2010-10-09T06:00:00.000000000', '2010-10-09T07:00:00.000000000',
       '2010-10-09T08:00:00.000000000', '2010-10-09T09:00:00.000000000',
       '2010-10-09T10:00:00.000000000', '2010-10-09T11:00:00.000000000',
       '2010-10-09T12:00:00.000000000', '2010-10-09T13:00:00.000000000',
       '2010-10-09T14:00:00.000000000', '2010-10-09T15:00:00.000000000',
       '2010-10-09T16:00:00.000000000', '2010-10-09T17:00:00.000000000',
       '2010-10-09T18:00:00.000000000', '2010-10-09T19:00:00.000000000',
       '2010-10-09T20:00:00.000000000', '2010-10-09T21:00:00.000000000',
       '2010-10-09T22:00:00.000000000', '2010-10-09T23:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time_counter  (time_counter) datetime64[ns] 2010-10-09 ... 2010-10-09T23:...
Attributes:
    axis:           T
    cell_methods:   time_counter: mean
    long_name:      Time axis
    standard_name:  time
    time_origin:    1950-01-01 00:00:00
    title:          Time

In [5]:
#checking specific dates that wer flagged on hrdps-archive on slack
# 2013m08d29 is fine
# hrdps2013m08d29 =xr.open_mfdataset(sorted(hrdps_path.glob(f"gemlam_y2013m08d29.nc")))
# hrdps2013m08d29.time_counter

#2007m02d10 <-feb was bad from feb 1 to 24 but now all fine
# hrdps2007m02d10 =xr.open_mfdataset(sorted(hrdps_path.glob(f"gemlam_y2007m02d10.nc")))
# hrdps2007m02d10.time_counter

#2008m07d15
hrdps2008m07d15 =xr.open_mfdataset(sorted(hrdps_path.glob(f"gemlam_y2008m07d15.nc")))
hrdps2008m07d15.time_counter

<xarray.DataArray 'time_counter' (time_counter: 24)>
array(['2008-07-15T00:00:00.000000000', '2008-07-15T01:00:00.000000000',
       '2008-07-15T02:00:00.000000000', '2008-07-15T03:00:00.000000000',
       '2008-07-15T04:00:00.000000000', '2008-07-15T05:00:00.000000000',
       '2008-07-15T06:00:00.000000000', '2008-07-15T07:00:00.000000000',
       '2008-07-15T08:00:00.000000000', '2008-07-15T09:00:00.000000000',
       '2008-07-15T10:00:00.000000000', '2008-07-15T11:00:00.000000000',
       '2008-07-15T12:00:00.000000000', '2008-07-15T13:00:00.000000000',
       '2008-07-15T14:00:00.000000000', '2008-07-15T15:00:00.000000000',
       '2008-07-15T16:00:00.000000000', '2008-07-15T17:00:00.000000000',
       '2008-07-15T18:00:00.000000000', '2008-07-15T19:00:00.000000000',
       '2008-07-15T20:00:00.000000000', '2008-07-15T21:00:00.000000000',
       '2008-07-15T22:00:00.000000000', '2008-07-15T23:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time_counter  (time_counter) datetime64[ns] 2008-07-15 ... 2008-07-15T23:...
Attributes:
    axis:           T
    cell_methods:   time_counter: mean
    long_name:      Time axis
    standard_name:  time
    time_origin:    1950-01-01 00:00:00
    title:          Time

In [3]:
start_year, end_year = 2008, 2014
years = range(start_year, end_year+1)

The PerformanceWarning messages and a substantial increase in execution time and memory footprint were due to the addition of the 3 variables LHTFL_surface, PRATE_surface, and RH_2maboveground to the HRDPS datasets on 5-Dec-2018. Adding those variables to the ones to drop resolved those issues.

In [5]:
drop_vars = (
    "precip", "qair", "solar", "therm_rad", "percentcloud", 
    "LHTFL_surface", "PRATE_surface", "RH_2maboveground",
)
hrdps = {}
t_total_start = time.time()
for year in years:
    print(f"start gathering {year}")
    t_start = time.time()
    hrdps[year] = xr.open_mfdataset(
        sorted(hrdps_path.glob(f"gemlam_y{year}*.nc")), #old naming convention from 2007-nov 2014
        drop_variables=drop_vars,
        #compat="override"
    )
    print(f"finished gathering {year}: {time.time() - t_start}s")
print(f"total gathering time: {time.time() - t_total_start}s")

start gathering 2008


/home/rbeutel/anaconda3/envs/eosc510/lib/python3.9/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/home/rbeutel/anaconda3/envs/eosc510/lib/python3.9/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[k

ValueError: cannot reindex or align along dimension 'time_counter' because the index has duplicate values

The dask optimization tips section in the xarray docs recommends doing .sel() and .isel() operations before resample() and groupby() ones. So, we will do that.<br>

Choosing to store the datasets we create at each step in new variables so that we can inspect them. They don't take up too much more memory because they are collections of dask arrays, not the actual results.

In [ ]:
hrdps_ssc = {
    year: hrdps[year].sel(x=slice(0, 480000))
    for year in years
}

Now we resample to get daily averages. Again, I am using a dict comprehension. This step does trigger some computation across all of the blocks of data that dask has divided things up into. So, it takes a bit of time, and I have wrapped it in timing code.

In [ ]:
t_start = time.time()
day_avgs = {
    year: hrdps_ssc[year].resample(time_counter="D").mean(dim="time_counter")
    for year in years
}
print(f"resampled to day averages: {time.time() - t_start}s")

Next, we concatenate the years together. Recall that day_avgs, like hrdps above is a dict whose keys are the year numbers, and whose values are xarray datasets. We get the the collection of datasets by calling the values() method on the dict. It's unfortunately confusing the method to do that has the same name as the method to access the NumPy array underlying an xarray.DataArray.

In [ ]:
hrdps_allyears = xr.concat(day_avgs.values(), dim="time_counter")

Now do a little bit of cleanup

In [ ]:
#remove dates after September 12, 2014 since those are already acounted for in the other netCDf file that we made
hrdps_allyears = hrdps_allyears.sel(time=slice('2016-01-01 12:00:00', '2019-12-31 12:00:00'))

In [11]:
# Drop the time_counter coordinate from nav_lat and nav_lon to make them 2d variables
hrdps_allyears["nav_lat"] = hrdps_allyears.nav_lat.sel(time_counter=hrdps_allyears.time_counter[0])
hrdps_allyears["nav_lon"] = hrdps_allyears.nav_lon.sel(time_counter=hrdps_allyears.time_counter[0])
#note how easy this is to do within an xarray instead of within multiple numpy arrays

In [12]:
# Restore that attribute metadata of the variables that got lost during resampling
for var in hrdps_allyears.data_vars:
    hrdps_allyears[var].attrs = hrdps[start_year][var].attrs

This is where the rubber hits the road!<br>

All of the above processing has been telling dask to add tasks to its processing graph but deferring the actual processing until we actually need to access its results (lazy processing). Doing things like accessing the underlying NumPy arrays in the dataset to plot them or look at a slice triggers processing. But it is best for large data collections for us to control when the processing is triggered, and how it is executed. We do that with the load() method on the dataset.<br>

For the kind of "concatenate and lightly process lots of netCDF datasets" workload in this notebook, telling dask to launch a collection of separate Python processes (workers) to distribute the tasks in its graph on to is usually the best approach. Here I chose to use 8 workers because I was running the notebook on salish while the nowcast-dev NEMO run was in progress. If top showed me that salish was not busy doing anything else I would have used up to 32 workers.

In [13]:
num_workers = 8
t_start = time.time()
hrdps_loaded = hrdps_allyears.load(scheduler="processes", num_workers=num_workers)
print(f"dask processing in {num_workers} processes to load result: {time.time() - t_start}s")

dask processing in 8 processes to load result: 614.5029768943787s


In [19]:
#add netCDF metadata
netcdf_title = 'HRDPSsubset.nc'
netcdf_comment = 'HRDPS2.5 dataset from September 12, 2014-Decmber 31,2020 used in CanRCM4 downscaling attempt for the Salish Sea'
notebook = 'LoadFiles.ipynb'

ds_attrs = {
        'creator_email':
            'rbeutel@eoas.ubc.ca',
        'institution_fullname': (
            'Earth, Ocean & Atmospheric Sciences,'
            ' University of British Columbia'
        ),
    'title': netcdf_title,
    'comment': netcdf_comment,
    'notebook': notebook,
    'summary': f'sea-level pressure, N/S wind, E/W wind, temperature',
    'history': (
            '[{}] File creation.'
            .format(dt.datetime.today().strftime('%Y-%m-%d'))
        )
}

#hrdps_loaded["attrs"]=ds_attrs #<- saying this is an invalid type and not sure why?
#this was a bit of an asside extra step to see how to do this, not a priority so just going to skip for now, ask Doug about this another time

In [23]:
hrdps_loaded

<xarray.Dataset>
Dimensions:       (time_counter: 2303, x: 193, y: 266)
Coordinates:
  * time_counter  (time_counter) datetime64[ns] 2014-09-12 ... 2020-12-31
  * x             (x) float64 0.0 2.5e+03 5e+03 ... 4.75e+05 4.775e+05 4.8e+05
  * y             (y) float64 0.0 2.5e+03 5e+03 ... 6.575e+05 6.6e+05 6.625e+05
Data variables:
    atmpres       (time_counter, y, x) float32 1.018e+05 1.018e+05 ... 1.013e+05
    nav_lat       (y, x) float64 45.66 45.67 45.67 45.68 ... 51.97 51.97 51.98
    nav_lon       (y, x) float64 233.0 233.0 233.1 233.1 ... 237.2 237.3 237.3
    tair          (time_counter, y, x) float32 291.4 291.4 291.4 ... 268.0 268.1
    u_wind        (time_counter, y, x) float32 -3.746 -3.735 ... 0.9753 1.008
    v_wind        (time_counter, y, x) float32 -9.562 -9.554 ... 0.6657 0.6766

Dump the daily averaged HRDPS fields and geo-coordinates to a netCDF so that they can be loaded with xarray.open_dataset() in the other processing steps without repeating the above processing.<br>

nc_file can be any path your want. The one below puts the file in the same directory as this notebook.<br>

NOTE: never commit this netCDF file to Git <br>
a) it can be regenerated by this code anyways<br>
b) its wayy too big

In [24]:
nc_file = Path("hrdps_day_avgs_postSep2014.nc")
hrdps_loaded.to_netcdf(nc_file, unlimited_dims=('time_counter'))